# Bringing Your Own Agent to NeMo Agent Toolkit

In this notebook, we'll show you how to integrate an existing agent with the NeMo Agent Toolkit (NAT).

You'll learn how to wrap agents from other frameworks so they work smoothly with NAT. This lets you take advantage of NAT features like MCP compatibility, observability, optimization, and profiling in your existing agent systems without refactoring your existing code.

# Table of Contents
- [0.0) Setup](#setup)
  - [0.1) Prerequisites](#prereqs)
  - [0.2) API Keys](#api-keys)
  - [0.3) Installing NeMo Agent Toolkit](#installing-nat)
- [1.0) Defining an 'Existing' Agent](#defining-existing-agent)
- [2.0) Existing Agent Migration](#migration)
  - [2.1) Migration Part 1: Transforming Your Existing Agent into a Workflow](#migration-part-1)
  - [2.2) Migration Part 2: Making Your Agent Configurable](#migration-part-2)
  - [2.3) Migration Part 3: Integration with NeMo Agent Toolkit](#migration-part-3)
  - [2.4) Migration Part 4: A Zero-Code Configuration](#migration-part-4)
- [3) Next Steps](#next-steps)

<span style="color:rgb(0, 31, 153); font-style: italic;">Note: In Google Colab use the Table of Contents tab to navigate.</span>

<a id="setup"></a>
# 0.0) Setup

<a id="prereqs"></a>
## 0.1) Prerequisites

- **Platform:** Linux, macOS, or Windows
- **Python:** version 3.11, 3.12, or 3.13
- **Python Packages:** `pip`

<a id="api-keys"></a>
## 0.2) API Keys

For this notebook, you will need the following API keys to run all examples end-to-end:

- **NVIDIA Build:** You can obtain an NVIDIA Build API Key by creating an [NVIDIA Build](https://build.nvidia.com) account and generating a key at https://build.nvidia.com/settings/api-keys
- **Tavily:** You can obtain a Tavily API Key by creating a [Tavily](https://www.tavily.com/) account and generating a key at https://app.tavily.com/home

Then you can run the cell below:

In [1]:
import getpass
import os

if "NVIDIA_API_KEY" not in os.environ:
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key

if "TAVILY_API_KEY" not in os.environ:
    tavily_api_key = getpass.getpass("Enter your Tavily API key: ")
    os.environ["TAVILY_API_KEY"] = tavily_api_key

<a id="installing-nat"></a>
## 0.3) Installing NeMo Agent Toolkit

The recommended way to install NAT is through `pip` or `uv pip`.

First, we will install `uv` which offers parallel downloads and faster dependency resolution.

In [2]:
!pip install uv

NeMo Agent toolkit can be installed through the PyPI `nvidia-nat` package.

There are several optional subpackages available for NAT. The `LangChain` subpackage contains useful components for integrating and running within [LangChain](https://python.langchain.com/docs/introduction/). Since LangChain will be used later in this notebook, let's install NAT with the optional `langchain` subpackage.

In [4]:
!uv pip install --pre "nvidia-nat[langchain]==1.3.0rc5"

Using Python 3.12.11 environment at: /Users/bbednarski/.venvs/unew_312
Resolved 147 packages in 237ms                                       
Uninstalled 3 packages in 83ms
Installed 3 packages in 104ms                               
 - authlib==1.6.4
 + authlib==1.6.5
 - nvidia-nat==1.3.0rc4.dev25+gfb32d84b (from file:///Users/bbednarski/Projects/nat-getting-started-fork/NeMo-Agent-Toolkit)
 + nvidia-nat==1.3.0rc5
 - nvidia-nat-langchain==1.4.0.dev12+gea18ede6 (from file:///Users/bbednarski/Projects/nat-getting-started-fork/NeMo-Agent-Toolkit/packages/nvidia_nat_langchain)
 + nvidia-nat-langchain==1.3.0rc5


<a id="defining-existing-agent"></a>
# 1.0) Defining an 'Existing' Agent

In this case study, we will use a simple, self-contained LangChain agent as a proxy for your 'existing' agent. This agent comes equipped with a search tool that is capable of retrieving context from the internet using the Tavily API. The cell below defines the simple LangChain agent with a string input query.

In [5]:
%%writefile langchain_agent.py
import os
import sys

from langchain import hub
from langchain.agents import AgentExecutor
from langchain.agents import create_react_agent
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_tavily import TavilySearch

def existing_agent_main():
    if len(sys.argv) < 2:
        print("Usage: python langchain_agent.py \"Your question here\"")
        sys.exit(1)
    user_input = sys.argv[1]

    # Initialize a tool to search the web
    search = TavilySearch(
        max_results=2,
        api_key=os.getenv("TAVILY_API_KEY")
    )

    # Initialize a LLM client
    llm = ChatNVIDIA(
        model_name="meta/llama-3.3-70b-instruct",
        temperature=0.0,
        max_completion_tokens=1024,
        api_key=os.getenv("NVIDIA_API_KEY")
    )

    # Use an open source prompt
    prompt = hub.pull("hwchase17/react-chat")

    # create tools list
    tools = [search]

    # Initialize a ReAct agent
    react_agent = create_react_agent(
        llm=llm,
        tools=tools,
        prompt=prompt,
        stop_sequence=["\nObservation"]
    )

    # Initialize an agent executor to iterate through reasoning steps
    agent_executor = AgentExecutor(
        agent=react_agent,
        tools=[search],
        max_iterations=15,
        handle_parsing_errors=True,
        verbose=True
    )

    # Invoke the agent with a user query
    response = agent_executor.invoke({"input": user_input, "chat_history": []})

    # Print the response
    print(response["output"])

if __name__ == "__main__":
    existing_agent_main()

Overwriting langchain_agent.py


There are three main components to this agent:

* a web search tool (Tavily)

* an LLM (Llama 3.3)

* an agent system prompt (obtained from the internet using `langchain.hub`)

The agent is constructed from these three components, then an _agent executor_ is created. Finally, we pass the requested input into the executor and get a response back.

All of the components in use come from LangGraph/LangChain, but any other framework or example could also work.

Next we will run this sample agent to validate that it works.

In [6]:
!python langchain_agent.py "Who won the last World Cup?"

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: tavily_search
Action Input: "last World Cup winner"{'query': 'last World Cup winner', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.topendsports.com/events/worldcupsoccer/winners.htm', 'title': 'FIFA World Cup Winners List - All Champions 1930-2022', 'content': 'Argentina won the most recent World Cup in 2022, defeating France in a thrilling final. **Argentina won the most recent World Cup in 2022**, defeating France in a thrilling final. * **Brazil** - 5 World Cups (1958, 1962, 1970, 1994, 2002) * **Germany** - 4 World Cups (1954, 1974, 1990, 2014) Brazil has won the most FIFA World Cups with 5 titles (1958, 1962, 1970, 1994, 2002). Argentina won the 2022 FIFA World Cup in Qatar, defeating France 4-

<a id="migration"></a>
# 2.0) Existing Agent Migration

<a id="migration-part-1"></a>
## 2.1) Migration Part 1: Transforming Your Existing Agent into a Workflow

NAT supports users bringing their own agent into the framework. As the primary entrypoint for agent execution is a NAT Workflow. For the first pass at NAT migration we will create a new workflow:

In [ ]:
!nat workflow create first_agent_attempt

Now that we've created a workflow directory for a new agent, we will continue by migrating the agent's functional code into the new workflow. In the next cell, we have adapted the agent code from the `def existing_agent_main()` into a new method `def first_agent_attempt_function()` which encapsulates the exact same functionality, but is decorated and registered for NAT workflow compatibility.

In [ ]:
%%writefile first_agent_attempt/src/first_agent_attempt/first_agent_attempt.py
import logging

from pydantic import Field

from nat.builder.builder import Builder
from nat.builder.framework_enum import LLMFrameworkEnum
from nat.builder.function_info import FunctionInfo
from nat.cli.register_workflow import register_function
from nat.data_models.function import FunctionBaseConfig

logger = logging.getLogger(__name__)


class FirstAgentAttemptFunctionConfig(FunctionBaseConfig, name="first_agent_attempt"):
    pass


@register_function(config_type=FirstAgentAttemptFunctionConfig, framework_wrappers=[LLMFrameworkEnum.LANGCHAIN])
async def first_agent_attempt_function(_config: FirstAgentAttemptFunctionConfig, _builder: Builder):
    import os

    from langchain import hub
    from langchain.agents import AgentExecutor
    from langchain.agents import create_react_agent
    from langchain_nvidia_ai_endpoints import ChatNVIDIA
    from langchain_tavily import TavilySearch

    # Initialize a tool to search the web
    search = TavilySearch(
        max_results=2,
        api_key=os.getenv("TAVILY_API_KEY")
    )

    # Initialize a LLM client
    llm = ChatNVIDIA(
        model_name="meta/llama-3.3-70b-instruct",
        temperature=0.0,
        max_completion_tokens=1024,
        api_key=os.getenv("NVIDIA_API_KEY")
    )

    # Use an open source prompt
    prompt = hub.pull("hwchase17/react-chat")

    # create tools list
    tools = [search]

    # Initialize a ReAct agent
    react_agent = create_react_agent(
        llm=llm,
        tools=tools,
        prompt=prompt,
        stop_sequence=["\nObservation"]
    )

    # Initialize an agent executor to iterate through reasoning steps
    agent_executor = AgentExecutor(
        agent=react_agent,
        tools=[search],
        max_iterations=15,
        handle_parsing_errors=True,
        verbose=True
    )

    async def _response_fn(input_message: str) -> str:
        response = agent_executor.invoke({"input": input_message, "chat_history": []})

        return response["output"]

    yield FunctionInfo.from_fn(_response_fn, description="A simple tool capable of basic internet search")

As you can see above, this is almost the exact same code as your 'existing' agent, but has been refactored to fit within a NAT function registration.

The only differences are 1) the definition of a closure function `_response_fn` which captures the instantiated agent executor and uses that to invoke the agent and return the response. And 2) the use of the @register_function decorator.

We can also simplify the workflow configuration from:

In [ ]:
%load first_agent_attempt/configs/config.yml


To:

In [ ]:
%%writefile first_agent_attempt/configs/config.yml
workflow:
  _type: first_agent_attempt

Then we can run the new workflow:

In [ ]:
!nat run --config_file first_agent_attempt/configs/config.yml --input "Who won the last World Cup?"

This first pass shows how little effort is required to bring an existing agent into NAT. But as we show in the next section, we can also extend this further to offer better configuration!

<a id="migration-part-2"></a>
## 2.2) Migration Part 2: Making Your Agent Configurable

For this next part, we will create another workflow, migrate similar functions as shown in Part 1, but make some important parameters configurable for the entire workflow.

In [ ]:
!nat workflow create second_agent_attempt

Then we can update the agent's function.

Below, we expand the configuration to include:

* the LLM it should use
* configurable values for iterations, verbosity, error handling
* an optional description


In [ ]:
%%writefile second_agent_attempt/src/second_agent_attempt/second_agent_attempt.py
import logging

from pydantic import Field

from nat.builder.builder import Builder
from nat.builder.framework_enum import LLMFrameworkEnum
from nat.builder.function_info import FunctionInfo
from nat.cli.register_workflow import register_function
from nat.data_models.component_ref import FunctionRef
from nat.data_models.component_ref import LLMRef
from nat.data_models.function import FunctionBaseConfig

logger = logging.getLogger(__name__)


class SecondAgentAttemptFunctionConfig(FunctionBaseConfig, name="second_agent_attempt"):
    llm_model_name: str = Field(description="LLM name to use")
    max_iterations: int = Field(default=15, description="Maximum number of iterations to run the agent")
    handle_parsing_errors: bool = Field(default=True, description="Whether to handle parsing errors")
    verbose: bool = Field(default=True, description="Whether to print verbose output")
    description: str = Field(default="", description="Description of the agent")


@register_function(config_type=SecondAgentAttemptFunctionConfig, framework_wrappers=[LLMFrameworkEnum.LANGCHAIN])
async def second_agent_attempt_function(config: SecondAgentAttemptFunctionConfig, builder: Builder):
    import os

    from langchain import hub
    from langchain.agents import AgentExecutor
    from langchain.agents import create_react_agent
    from langchain_nvidia_ai_endpoints import ChatNVIDIA
    from langchain_tavily import TavilySearch

    # Initialize a tool to search the web
    search = TavilySearch(
        max_results=2,
        api_key=os.getenv("TAVILY_API_KEY")
    )

    # Initialize a LLM client
    llm = ChatNVIDIA(
        model_name=config.llm_model_name,
        temperature=0.0,
        max_completion_tokens=1024,
        api_key=os.getenv("NVIDIA_API_KEY")
    )

    # Use an open source prompt
    prompt = hub.pull("hwchase17/react-chat")

    # create tools list
    tools = [search]

    # Initialize a ReAct agent
    react_agent = create_react_agent(
        llm=llm,
        tools=tools,
        prompt=prompt,
        stop_sequence=["\nObservation"]
    )

    # Initialize an agent executor to iterate through reasoning steps
    agent_executor = AgentExecutor(
        agent=react_agent,
        tools=[search],
        **config.model_dump(include={"max_iterations", "handle_parsing_errors", "verbose"})
    )

    async def _response_fn(input_message: str) -> str:
        response = agent_executor.invoke({"input": input_message, "chat_history": []})

        return response["output"]

    yield FunctionInfo.from_fn(_response_fn, description=config.description)

We can then update the configuration file to include the configuration options which previously were embedded into the agent's code:

In [ ]:
%%writefile second_agent_attempt/configs/config.yml
workflow:
  _type: second_agent_attempt
  llm_model_name: meta/llama-3.3-70b-instruct
  max_iterations: 15
  verbose: false
  description: "A helpful assistant that can search the internet for information"

We can then run this modified agent to demonstrate the YAML configuration capabilities of NeMo Agent toolkit.

In [ ]:
!nat run --config_file second_agent_attempt/configs/config.yml --input "Who won the last World Cup?"

<a id="migration-part-3"></a>
## 2.3) Migration Part 3: Integration with NeMo Agent Toolkit

NeMo Agent toolkit comes with support for various LLM Providers, Frameworks, and additional components.

For this last part of migrating an agent, we will adapt the agent to use built-in toolkit components rather than importing directly from LangChain.

Changes made below:
- changing from LLM model name to an LLM _reference_
- adapting the code to query NAT for the LLM and Tools to use
- switching to the built-in Tavily Search Tool

In [ ]:
!nat workflow create third_agent_attempt

In [ ]:
%%writefile third_agent_attempt/src/third_agent_attempt/third_agent_attempt.py
import logging

from pydantic import Field

from nat.builder.builder import Builder
from nat.builder.framework_enum import LLMFrameworkEnum
from nat.builder.function_info import FunctionInfo
from nat.cli.register_workflow import register_function
from nat.data_models.component_ref import FunctionRef
from nat.data_models.component_ref import LLMRef
from nat.data_models.function import FunctionBaseConfig

logger = logging.getLogger(__name__)


class ThirdAgentAttemptFunctionConfig(FunctionBaseConfig, name="third_agent_attempt"):
    tool_names: list[FunctionRef] = Field(default_factory=list, description="List of tool names to use")
    llm_name: LLMRef = Field(description="LLM name to use")
    max_iterations: int = Field(default=15, description="Maximum number of iterations to run the agent")
    handle_parsing_errors: bool = Field(default=True, description="Whether to handle parsing errors")
    verbose: bool = Field(default=True, description="Whether to print verbose output")
    description: str = Field(default="", description="Description of the agent")

# Since our agent relies on Langchain, we must explicitly list the supported framework wrappers.
# Otherwise, the toolkit would not know the correct type to return from the builder

@register_function(config_type=ThirdAgentAttemptFunctionConfig, framework_wrappers=[LLMFrameworkEnum.LANGCHAIN])
async def third_agent_attempt_function(config: ThirdAgentAttemptFunctionConfig, builder: Builder):
    import os

    from langchain import hub
    from langchain.agents import AgentExecutor
    from langchain.agents import create_react_agent

    # Create a list of tools for the agent
    tools = await builder.get_tools(config.tool_names, wrapper_type=LLMFrameworkEnum.LANGCHAIN)

    llm = await builder.get_llm(config.llm_name, wrapper_type=LLMFrameworkEnum.LANGCHAIN)

    # Use an open source prompt
    prompt = hub.pull("hwchase17/react-chat")

    # Initialize a ReAct agent
    react_agent = create_react_agent(
        llm=llm,
        tools=tools,
        prompt=prompt,
        stop_sequence=["\nObservation"]
    )

    # Initialize an agent executor to iterate through reasoning steps
    agent_executor = AgentExecutor(
        agent=react_agent,
        tools=tools,
        **config.model_dump(include={"max_iterations", "handle_parsing_errors", "verbose"})
    )

    async def _response_fn(input_message: str) -> str:
        response = agent_executor.invoke({"input": input_message, "chat_history": []})

        return response["output"]

    yield FunctionInfo.from_fn(_response_fn)

We can then update the configuration file to include LLM and Function definitions that before were embedded into the agent's code:

In [ ]:
%%writefile third_agent_attempt/configs/config.yml
llms:
  nim_llm:
    _type: nim
    model_name: meta/llama-3.3-70b-instruct
    temperature: 0.0
    max_tokens: 1024
    api_key: $NVIDIA_API_KEY

functions:
  search:
    _type: tavily_internet_search
    max_results: 2
    api_key: $TAVILY_API_KEY

workflow:
  _type: third_agent_attempt
  tool_names: [search]
  llm_name: nim_llm
  max_iterations: 15
  verbose: false
  description: "A helpful assistant that can search the internet for information"

Finally, we can run this modified agent to demonstrate the flexibility and adaptiveness of using NeMo Agent toolkit.

In [ ]:
!nat run --config_file third_agent_attempt/configs/config.yml --input "Who won the last World Cup?"

<a id="migration-part-4"></a>
## 2.4) Migration Part 4: A Zero-Code Configuration

Sometimes NeMo Agent toolkit has all of the components you need already. In cases like these, we can rely on zero code additions. The effect of this is being able to **only** specify a configuration file, demonstrating the power of a batteries-included approach.

The required components for this base example were:
- An LLM (NVIDIA NIM-based)
- Tavily Internet Search Tool
- ReAct Agent

In [ ]:
%%writefile search_agent.yml
llms:
  nim_llm:
    _type: nim
    model_name: meta/llama-3.3-70b-instruct
    temperature: 0.0
    max_tokens: 1024
    api_key: $NVIDIA_API_KEY

functions:
  search:
    _type: tavily_internet_search
    max_results: 2
    api_key: $TAVILY_API_KEY

workflow:
  _type: react_agent
  tool_names: [search]
  llm_name: nim_llm
  verbose: false
  description: "A helpful assistant that can search the internet for information"

In [ ]:
!nat run --config_file search_agent.yml --input "Who won the last World Cup?"

This concludes the "Bringing Your Own Agent to NeMo Agent toolkit" notebook.

Throughout this notebook, we've demonstrated a complete migration journey from a standalone agent to a fully integrated NeMo Agent Toolkit workflow. In **Part 1**, we started with a basic ReAct agent using raw API calls and manual prompt engineering. In **Part 2**, we refactored the code to use NAT's built-in components, replacing custom implementations with standardized LLM and tool abstractions. In **Part 3**, we elevated the architecture by leveraging NAT's ReAct agent implementation, eliminating the need for custom agent logic entirely. Finally, in **Part 4**, we achieved a zero-code solution using only a YAML configuration file, demonstrating NAT's batteries-included philosophy.

By migrating existing agents to NeMo Agent Toolkit, you gain access to a unified platform that standardizes how agents are built, evaluated, and deployed. This migration unlocks powerful capabilities: consistent evaluation frameworks for comparing agent performance across different implementations, systematic optimization through shared tooling and best practices, and comprehensive observability with built-in logging and monitoring. Rather than maintaining custom evaluation scripts, optimization pipelines, and monitoring solutions for each agent, NAT provides these capabilities out of the box, allowing you to focus on solving business problems rather than reinventing infrastructure. This unified approach not only accelerates development but also ensures reproducibility and maintainability across your entire agent ecosystem.




<a id="next-steps"></a>
# 3.0) Next Steps

Next, look at "Adding Tools to NeMo Agent Toolkit Agents" where you will interactively learn how to create your own tools and agents with NAT.